In [2]:
import sys
import os
sys.path.append("/Users/baeyeongmin/Desktop/workspace/dacon-2024-gbt-hackerton")
sys.path.append("/root/dacon-2024-gbt-hackerton")

In [3]:
import os
import wandb
import torch
import pandas as pd
from src.config import CFG
from src.data_processing import load_data, preprocess_data, get_label_encoded, split_data
from src.models import get_model_and_tokenizer
from src.datasets import TextDataset
from src.training import train_model
from src.inference import get_test_predictions
from src import root_path
from src.utils import get_sample_submission
from torch.utils.data import DataLoader

CFG.model_type = "primary"

In [9]:
from src.config import CFG
from src.data_processing import get_label_encoded, load_data, preprocess_data
from src import root_path
import torch

from src.models import get_model_and_tokenizer


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_df, test_df = load_data(root_path=root_path)
test_df = preprocess_data(test_df)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
test_dataset = TextDataset(test_df.text.tolist(), None, tokenizer, max_len=CFG.tokenizer_max_len)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=CFG.num_workers, pin_memory=CFG.pin_memory)
    
_, primary_label_encoder = get_label_encoded(train_df, model_type='primary')
_, secondary_label_encoder = get_label_encoded(train_df, model_type='secondary')
primary_label_decoder = {i: label for label, i in primary_label_encoder.items()}
secondary_label_decoder = {i: label for label, i in secondary_label_encoder.items()}
primary_model, p_tokenizer = get_model_and_tokenizer(CFG.model_name, len(primary_label_encoder), CFG.attention_type)
secondary_model, s_tokenizer = get_model_and_tokenizer(CFG.model_name, len(secondary_label_encoder), CFG.attention_type)

# TODO: wandb artifact에서 가져오는 코드로 변경.
primary_model_filepath = f"{root_path}/primary_best_model.pth"
primary_model.load_state_dict(torch.load(primary_model_filepath, map_location=device))

# secondary_model_filepath = f"{root_path}/secondary_best_model.pth"
# secondary_model.load_state_dict(torch.load(secondary_model_filepath, map_location=device))
# secondary_model.eval()

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [21]:
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=CFG.num_workers, pin_memory=CFG.pin_memory)

In [11]:
primary_model.to(device)
primary_model.eval()


BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [22]:
from tqdm import tqdm
from sklearn.metrics import f1_score
def evaluate_model(model, val_loader, device):
    model.eval()
    pri_predictions = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Combining'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            pri_predictions.extend(preds.cpu().tolist())
    return pri_predictions


In [23]:
primary_label_decoder

{0: '비지역', 1: '지역'}

In [24]:
secondary_label_decoder

{0: '문화:전시_공연',
 1: '국제',
 2: '정치:선거',
 3: '경제:금융_재테크',
 4: '사회:의료_건강',
 5: '정치:행정_자치',
 6: '정치:국회_정당',
 7: '스포츠:축구',
 8: '경제:서비스_쇼핑',
 9: '경제:부동산',
 10: '사회:교육_시험',
 11: '사회:미디어',
 12: 'IT_과학:인터넷_SNS',
 13: '사회:장애인',
 14: '사회:노동_복지',
 15: '경제:경제일반',
 16: '정치:정치일반',
 17: '사회:사회일반',
 18: '문화:문화일반',
 19: '경제:취업_창업',
 20: '사회:사건_사고',
 21: '스포츠:올림픽_아시안게임',
 22: '사회:환경',
 23: '정치:외교',
 24: '경제:산업_기업',
 25: '스포츠:골프',
 26: '경제:유통',
 27: 'IT_과학:모바일',
 28: '사회:여성',
 29: '사회:날씨',
 30: '문화:방송_연예',
 31: 'IT_과학:IT_과학일반',
 32: '경제:반도체',
 33: '경제:자원',
 34: '문화:종교',
 35: '경제:자동차',
 36: '경제:무역',
 37: 'IT_과학:콘텐츠',
 38: '경제:증권_증시',
 39: 'IT_과학:과학',
 40: '경제:외환',
 41: '문화:요리_여행',
 42: '정치:청와대',
 43: '문화:출판',
 44: '문화:미술_건축',
 45: '문화:음악',
 46: '스포츠:농구_배구',
 47: '문화:생활',
 48: '정치:북한',
 49: '스포츠:야구',
 50: '문화:학술_문화재',
 51: 'IT_과학:보안',
 52: '문화:영화',
 53: '스포츠:월드컵',
 54: '스포츠:스포츠일반'}

In [25]:
pri_predictions = evaluate_model(primary_model, test_loader, device)

Combining: 100%|██████████| 46/46 [01:11<00:00,  1.56s/it]


In [30]:
secondary_model_filepath = f"{root_path}/secondary_best_model.pth"
secondary_model.load_state_dict(torch.load(secondary_model_filepath, map_location=device))
secondary_model.to(device)
secondary_model.eval()

BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [35]:
from pprint import pprint
pprint(pri_predictions[:10])


[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]


In [36]:
se_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Combining'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = secondary_model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        se_predictions.extend(preds.cpu().tolist())

Combining: 100%|██████████| 46/46 [01:11<00:00,  1.55s/it]


In [27]:
len(pri_predictions)

23405

In [37]:
len(se_predictions)

23405

In [49]:
result = []
for p, s in zip(pri_predictions, se_predictions):
    if p == 0:
        if s == 1:
            result.append(55) # '국제'
        else:
            result.append(s)
    else:
        result.append(p)

In [51]:
secondary_label_decoder[55] = '국제'
secondary_label_decoder[1] = '지역'

In [52]:
secondary_label_decoder

{0: '문화:전시_공연', 1: '지역', 2: '정치:선거', 3: '경제:금융_재테크', 4: '사회:의료_건강', 5: '정치:행정_자치', 6: '정치:국회_정당', 7: '스포츠:축구', 8: '경제:서비스_쇼핑', 9: '경제:부동산', 10: '사회:교육_시험', 11: '사회:미디어', 12: 'IT_과학:인터넷_SNS', 13: '사회:장애인', 14: '사회:노동_복지', 15: '경제:경제일반', 16: '정치:정치일반', 17: '사회:사회일반', 18: '문화:문화일반', 19: '경제:취업_창업', 20: '사회:사건_사고', 21: '스포츠:올림픽_아시안게임', 22: '사회:환경', 23: '정치:외교', 24: '경제:산업_기업', 25: '스포츠:골프', 26: '경제:유통', 27: 'IT_과학:모바일', 28: '사회:여성', 29: '사회:날씨', 30: '문화:방송_연예', 31: 'IT_과학:IT_과학일반', 32: '경제:반도체', 33: '경제:자원', 34: '문화:종교', 35: '경제:자동차', 36: '경제:무역', 37: 'IT_과학:콘텐츠', 38: '경제:증권_증시', 39: 'IT_과학:과학', 40: '경제:외환', 41: '문화:요리_여행', 42: '정치:청와대', 43: '문화:출판', 44: '문화:미술_건축', 45: '문화:음악', 46: '스포츠:농구_배구', 47: '문화:생활', 48: '정치:북한', 49: '스포츠:야구', 50: '문화:학술_문화재', 51: 'IT_과학:보안', 52: '문화:영화', 53: '스포츠:월드컵', 54: '스포츠:스포츠일반', 55: '국제'}

In [53]:

decoded_predictions = [secondary_label_decoder[pred] for pred in result]

In [54]:
decoded_predictions

['사회:사회일반', '사회:의료_건강', '정치:행정_자치', '경제:취업_창업', '지역', '경제:반도체', '지역', '국제', '사회:사건_사고', '경제:자원', '경제:부동산', '정치:국회_정당', '지역', '사회:여성', 'IT_과학:인터넷_SNS', '문화:음악', '사회:사회일반', '사회:여성', '지역', 'IT_과학:과학', '문화:종교', '지역', '지역', '지역', '지역', '지역', '지역', '지역', '스포츠:골프', '정치:국회_정당', '지역', '지역', '지역', '경제:부동산', '지역', '지역', '지역', '문화:생활', '경제:취업_창업', '문화:방송_연예', '지역', '지역', '지역', '스포츠:축구', '지역', '사회:사건_사고', '사회:장애인', '지역', '스포츠:올림픽_아시안게임', '지역', '지역', 'IT_과학:인터넷_SNS', '경제:서비스_쇼핑', '지역', '지역', '지역', '지역', '지역', '경제:경제일반', '지역', '지역', '경제:취업_창업', '정치:국회_정당', '사회:사건_사고', '정치:행정_자치', '정치:국회_정당', 'IT_과학:모바일', '지역', '지역', '경제:취업_창업', '지역', '경제:부동산', '사회:사건_사고', '지역', '경제:반도체', '지역', '지역', '지역', '사회:교육_시험', '경제:서비스_쇼핑', '스포츠:올림픽_아시안게임', '지역', '지역', '지역', '지역', '지역', '지역', '정치:행정_자치', '정치:청와대', '경제:부동산', '사회:사회일반', '경제:자동차', '지역', '정치:행정_자치', '사회:환경', '지역', '지역', '경제:산업_기업', '정치:국회_정당', '지역', '지역', '지역', '사회:사건_사고', '지역', '지역', '경제:부동산', '사회:교육_시험', '지역', '경제:무역', '지역', '지역', '지역', '문화:요리_여행', '경제:반도체', '지역'

In [44]:
len(result)/len(se_predictions)

0.5125400555436872

In [55]:
sample_submission = pd.read_csv(os.path.join(root_path, "./sample_submission.csv"))
sample_submission["분류"] = decoded_predictions
submission_filepath = f"./combined.csv"
sample_submission.to_csv(submission_filepath, encoding='UTF-8-sig', index=False)

In [56]:
from dacon_submit_api import dacon_submit_api 

dacon_token = 'd5ea5ec7e519de6412291ab093463dc54315baa080104aeee57ae0ab51543149'
result = dacon_submit_api.post_submission_file(
submission_filepath, 
dacon_token, 
'236372', 
'김밥조아', 
'submission 메모 내용' )

{'isSubmitted': True, 'detail': 'Success'}
